In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Layer, Flatten, BatchNormalization, Conv2D,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


import os

# Path dataset
test_folder = "test"
train_folder = "train"
valid_folder = "valid"

# Result path
result_path = f"run"
os.makedirs(result_path, exist_ok=True)

checkpoint_path = os.path.join(result_path, "best_model.h5")
loss_image_path = os.path.join(result_path, 'validation loss.png')
acc_image_path = os.path.join(result_path, 'validation accuracy.png')

#### Hyperparameter
image_size = (224, 224)
batch_size = 32
epochs = 50
learning_rate = 0.0001

#https://www.kaggle.com/datasets/gpiosenka/sports-classification/code
# Load the CSV file
df = pd.read_csv('sports.csv')

# Filter based on the data set column
train_df = df[df['data set'] == 'train']
valid_df = df[df['data set'] == 'valid']
test_df = df[df['data set'] == 'test']

# Extract filepaths and labels
X_train_paths = train_df['filepaths'].values
y_train_labels = train_df['class id'].values
X_val_paths = valid_df['filepaths'].values
y_val_labels = valid_df['class id'].values
X_test_paths = test_df['filepaths'].values
y_test_labels = test_df['class id'].values

# Assuming y_train_labels and y_val_labels are your labels and num_classes is the number of classes.
y_train = to_categorical(y_train_labels, num_classes=100)
y_val = to_categorical(y_val_labels, num_classes=100)
y_test = to_categorical(y_test_labels, num_classes=100)

# Function to load and preprocess each image
def load_and_preprocess_image(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img /= 255.0  # normalize to [0,1] range
    return img, label

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_paths, y_train))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_paths, y_val))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test_paths, y_test))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(batch_size)



2023-09-28 16:27:32.962306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 16:27:33.473102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-09-28 16:27:34.537878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L3

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
# ... Your existing imports and code ...
import tensorflow as tf
class BasicBlock(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.residual = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=stride,
                                            padding="same")

    def call(self, inputs, training=None, **kwargs):
        residual = self.residual(inputs)
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.keras.layers.Add()([residual, x])
        x = tf.nn.relu(x)
        return x

class PINet18(tf.keras.models.Model):
    def __init__(self, block_list, initial_filters=16, num_classes=100, **kwargs):
        super(PINet18, self).__init__(**kwargs)
        self.num_blocks = len(block_list)
        self.block_list = block_list
        self.out_filters = initial_filters
        self.conv1 = tf.keras.layers.Conv2D(filters=self.out_filters,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.blocks = tf.keras.models.Sequential()

        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):
                if block_id != 0 and layer_id == 0:
                    block = BasicBlock(self.out_filters, stride=2)
                else:
                    block = BasicBlock(self.out_filters, stride=1)
                self.blocks.add(block)
            self.out_filters *= 2

        self.final_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=num_classes)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.blocks(x, training=training)
        x = self.final_pool(x)
        x = self.fc(x)
        return x


model = PINet18(block_list=[2, 2, 2, 2])  # for PiNet18


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.build(input_shape=(None, 224, 224, 3))
# model.summary()

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
)

# ... Your existing code for testing and evaluation ...




Epoch 1/50
422/422 [==============================] - 45s 86ms/step - loss: 8.5197 - accuracy: 0.0071 - val_loss: 8.4262 - val_accuracy: 0.0100
Epoch 2/50
422/422 [==============================] - 35s 83ms/step - loss: 9.0625 - accuracy: 0.0059 - val_loss: 8.1617 - val_accuracy: 0.0100
Epoch 3/50
422/422 [==============================] - 35s 83ms/step - loss: 8.5251 - accuracy: 0.0090 - val_loss: 8.7572 - val_accuracy: 0.0020
Epoch 4/50
422/422 [==============================] - 35s 83ms/step - loss: 8.6626 - accuracy: 0.0050 - val_loss: 7.9653 - val_accuracy: 0.0120
Epoch 5/50
422/422 [==============================] - 35s 84ms/step - loss: 8.7262 - accuracy: 0.0082 - val_loss: 9.0227 - val_accuracy: 0.0080
Epoch 6/50
422/422 [==============================] - 36s 84ms/step - loss: 7.8187 - accuracy: 0.0096 - val_loss: 8.0366 - val_accuracy: 0.0040
Epoch 7/50
422/422 [==============================] - 36s 84ms/step - loss: 8.6551 - accuracy: 0.0072 - val_loss: 8.6081 - val_accuracy: